**WARNING**: This is a legacy notebook from another project and was kept to record how the examples were colelcted, but is not runnable.

# Download the latest copy of the dataset
Get the latest copy of the data from MongoDB. 

In [1]:
from hydronet.db import HydroNetDB, HydroNetRecord
from tqdm import tqdm
import pandas as pd

Configuration

In [2]:
n_random = 2
n_best = 2

## Get an example of cluster of each size
Pick some based on low-energy minimia and some randomly

In [4]:
mongo = HydroNetDB.from_connection_info(hostname=hostname, password=password, username=username)
n_docs = mongo.collection.estimated_document_count()
print(f'Connected to a database with around {n_docs} water clusters')

Connected to a database with around 4585608 water clusters


Collect the lowest-energy structures

In [5]:
selected = {}  # Dictionary to prevent duplicates

In [6]:
for csize in tqdm(range(4, 30, 6)):
    for cluster in mongo.collection.find({'n_waters': csize}, sort=[('energy', 1)], limit=n_best):
        record = HydroNetRecord.parse_obj(cluster)
        selected[cluster['_id']] = record

100%|██████████| 5/5 [02:42<00:00, 32.51s/it]


Collect some structures at random

In [7]:
for csize in tqdm(range(4, 30, 6)):
    for cluster in mongo.collection.find({'n_waters': csize}, sort=[('position', 1)], limit=n_random):
        record = HydroNetRecord.parse_obj(cluster)
        selected[cluster['_id']] = record

100%|██████████| 5/5 [00:12<00:00,  2.53s/it]


Prepare them for output. We need the ID, XYZ geometry, ring hash, and TTM energy. 

In [8]:
output = []
for cid, cluster in selected.items():
    output.append({
        'id': cid,
        'n_waters': cluster.n_waters,
        'xyz': cluster.xyz,
        'cycle_hash': cluster.cycle_hash,
        'ttm_energy': cluster.energy
    })
output = pd.DataFrame(output).sort_values(['n_waters', 'ttm_energy'])
output.to_csv('example_structures.csv', index=False)